# 3章word2vec

前章から引き続き、本章のテーマも単語の分散表現です。前章では、「カウントベー
スの手法」によって単語の分散表現を得ました。本章では、「カウントベースの手法」
に代わる強力な手法として「推論ベースの手法」を見ていきます。
「推論ベースの手法」は、その名前が示すとおり、推論をする手法です。もちろん、
その推論にはニューラルネットワークが使えます。そして、ここで有名なword2vec
が登場します。本章では、word2vec の仕組みをじっくりと時間をかけて見ていき、
それを実装することで理解を確かなものとします。本章の目標は、“シンプル” なword2vec を実装することです。このシンプルなword2vec では、処理効率は犠牲にして、分かりやすさを優先しています。そのため、大きなデータセットは扱えませんが、小さなデータセットであれば問題なく処理できます。次章では、本章のシンプルなword2vec にいくつかの改良を加え、“本物” の
word2vecを完成させます。それでは、推論ベースの手法へ、word2vec の世界へ進みましょう！

## 3.1 推論ベースの手法とニューラルネットワーク

単語をベクトルで表す研究は、これまで盛んに行われてきました。その中でも成功を収めた手法を見ていくと、それらは大きく2 つに分けられます。ひとつは「カウントベースの手法」、もうひとつは「推論ベースの手法」です。単語の意味を獲得するためのアプローチは両者で大きく異なりますが、その背景には両者ともに分布仮説があります。ここでは、カウントベースの手法の問題点を指摘し、それに代わる推論ベースの手法の利点を大きな視点で説明します。そして、word2vec への下準備を行うために、ニューラルネットワークで「単語」を処理する例を見ていきます。

### 3.1.1 カウントベースの手法の問題点

これまで見てきたように、カウントベースの手法では、周囲の単語の頻度によって
単語を表現しました。具体的には、単語の共起行列を作り、その行列に対してSVD
を適用することで、密なベクトル――単語の分散表現――を獲得したのです。しか
し、カウントベースの手法には問題があります。その問題は、大規模なコーパスを扱
う場合に発生します。現実的には、コーパスで扱う語彙数は非常に巨大になりえます。たとえば、英語の語彙数は100万をゆうに超えると言われています。語彙数を100 万とした場合、カウントベースの手法では、100 万×100 万の巨大な行列を作ることになります。しかし、そのような巨大な行列に対してSVD を行うことは、現実的ではありません。

カウントベースの手法は、コーパス全体の統計データ（共起行列やPPMI など）を利用して、1 回の処理（SVD など）で単語の分散表現を獲得します。一方、推論ベースの手法では、たとえばニューラルネットワークを用いる場合は、ミニバッチで学習するのが一般的です。ミニバッチで学習するとは、ニューラルネットワークは一度に少量（ミニバッチ）の学習サンプルを見ながら、重みを繰り返し更新するということです。この学習における枠組みの違いは、図で表すと図3-1 のようになります。

![3_1](fig2/3_1.png)

図3-1 に示すように、カウントベースの手法は、学習データを一度にまとめて処理します。それに対して、推論ベースの手法では、学習データの一部を使って逐次的に学習します。これが意味することは、語彙数が大きいコーパスにおいてSVD などの
計算量が膨大で処理が難しい場合でも、ニューラルネットワークではデータを小分けにして学習できるということです。さらに、ニューラルネットワークの学習は複数マシン/複数GPU の利用による並列計算も可能であり、全体の学習も高速化できます。
この点において、推論ベースの手法に分があります。この他にも、推論ベースの手法がカウントベースの手法よりも魅力的な点があります。その点については、推論ベースの手法について（特にword2vec について）詳しく説明した後に、「3.5.3 カウントベースv.s. 推論ベース」で、もう一度議論したいと思います。

### 3.1.2 推論ベースの手法の概要

推論ベースの手法では、「推論」することが主な作業になります。これは、図3-2 で
示すように、周囲の単語（コンテキスト）が与えられたときに、「？」にどのような
単語が出現するのかを推測する作業です。

![3_2](fig2/3_2.png)

図3-2 のような推論問題を解くこと、そして、学習することが「推論ベースの手法」
の扱う問題です。このような推論問題を繰り返し解くことで、単語の出現パターンを学習します。このとき“モデル視点” に立つと、この推論問題は図3-3 のように見え
ます。

![3_3](fig2/3_3.png)

図3-3 に示すように、推論ベースの手法では、何らかのモデルが登場します。私たちは、そのモデルにニューラルネットワークを使います。モデルはコンテキスト情報を入力として受け取り、（出現しうるであろう）各単語の出現する確率を出力します。
そのような枠組みの中で、正しい推測ができるように、コーパスを使ってモデルの学習を行います。そして、その学習の結果として、単語の分散表現を得られるというのが推論ベースの手法の全体図になります。

### 3.1.3 ニューラルネットワークにおける単語の処理方法

私たちはこれからニューラルネットワークを使って「単語」を処理します。しかし、
ニューラルネットワークは“you” や“say” などの単語をそのままでは処理できませ
ん。ニューラルネットワークで単語を処理するには、それを「固定長のベクトル」に
変換する必要があります。そのための方法のひとつは、単語をone-hot 表現（または
one-hot ベクトル）へと変換することです。one-hot 表現とは、ベクトルの要素の中
でひとつだけが1 で、残りはすべて0 であるようなベクトルを言います。
one-hot 表現について具体的に見ていきましょう。ここでは前章と同じく、「You say goodbye and I say hello.」という1 文をコーパスとして扱うものとして話を進め
ます。このコーパスでは、語彙が全部で7 個存在します（“you”, “say”, “goodbye”,
“and”, “i”, “hello”, “.”）。このとき各単語は図3-4 のようにone-hot 表現へ変換す
ることができます。

![3_4](fig2/3_4.png)

図3-4 に示すように単語は、テキスト、単語ID、そしてone-hot 表現でそれぞれ
表現できます。このとき単語をone-hot 表現に変換するには、語彙数分の要素を持つ
ベクトルを用意して、単語ID の該当する箇所を1 に、残りはすべて0 に設定します。
このように単語を固定長のベクトルに変換してしまえば、私たちのニューラルネット
ワークの入力層は、図3-5 のようにニューロンの数を“固定” することができます。

![3_5](fig2/3_5.png)

図3-5 に示すように、入力層は7 つのニューロンによって表されます。このとき、7つのニューロンはそれぞれ7 つの単語に対応します（ひとつ目のニューロンは「you」
に、2 つ目のニューロンは「say」といったように）。
これで話は単純になりました。なぜなら単語をベクトルで表すことができれば、そ
のベクトルはニューラルネットワークを構成するさまざまな「レイヤ」によって処理
することができるからです。たとえば、one-hot 表現で表されたひとつの単語に対し
て、全結合層で変換する場合は図3-6 のように書くことができます。

![3_6](fig2/3_6.png)

全結合層ということで、図3-6 に示すように、すべてのノードには矢印によるつな
がりがあります。この矢印には重み（パラメータ）が存在し、入力層のニューロンと
の重み付き和が中間層のニューロンとなります。なお、本章で使用する全結合層では
バイアスは省略することにします（これは次のword2vec の説明を見越してのこと
です）。さて、図3-6 ではニューロン間の結びつきを矢印によって図示しましたが、これ以
降は、重みを明確に示すため図3-7 のような図法を用いることにします。

![3_7](fig2/3_7.png)

それでは、ここまでの話をコードベースで見ていきましょう。早速ですが、ここでの全結合層による変換は、Python で次のように書くことができます。

```python
import numpy as np
c = np.array([[1, 0, 0, 0, 0, 0, 0]]) # 入力
W = np.random.randn(7, 3) # 重み
h = np.dot(c, W) # 中間ノード
print(h)
 [[-0.70012195 0.25204755 -0.79774592]]
```

このコード例では、単語ID が0 の単語をone-hot 表現で表し、それを全結合層に
よって変換する例を示しています。復習になりますが、全結合層の計算は行列の積に
よって行うことができました。そしてそれはNumPy のnp.dot() で実装できます
（バイアスは省略）。

さて、上のコードで注目してほしいのは、c とW の行列の積の箇所です。ここで、
c はone-hot 表現であるため、単語ID に対応する要素が1 で、それ以外は0 である
ベクトルになります。そのため、上のコードのc とW の行列の積で行っていることは、図3-8 に示すように、重みの行ベクトルを“抜き出す” ことに相当します。

![3_8](fig2/3_8.png)

ここで、重みから行ベクトルを抜き出すためだけに行列の積を計算するのは非効率
に感じられるかもしれません。その点については、「4.1 word2vec の改良①」で改良
を行う予定です。なお、上のコードで行ったことは、（1 章で実装済みの）MatMul
レイヤによっても行うことができます。これは次のようなコードになります。

```python
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul
c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)
# [[-0.70012195 0.25204755 -0.79774592]]
```

ここでは、common ディレクトリにあるMatMul レイヤをインポートして利用します。後は、MatMul レイヤに重みW を設定し、forward() メソッドによって順伝播の処理を行います。